<a href='https://ai.meng.duke.edu'> = <img align="left" style="padding-top:10px;" src=https://storage.googleapis.com/aipi_datasets/Duke-AIPI-Logo.png>

---

In [1]:
NAME = "AIPI Project"
TEAM_MEMBER = "Chen Bian" "Asmita Dalai", "Yue Zhang", "Yufei Lei"

In [2]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
from shapely.geometry import Point, Polygon

# Chen Bian's coding part

# Part 1: Data Preparation

We have three datasets in the form of CSV. They are supermarket location and detailed information from Durham opendata source(https://live-durhamnc.opendata.arcgis.com/datasets/DurhamNC::supermarkets-warehouse-clubs-and-supercenters-1/about), fastfood location in America from http://www.fastfoodmaps.com/, and the apartment location dataset in Durham which is collected by us from https://www.apartments.com/.

All the common features of these three dataset are that they all have specific latitude and longitude. So, we can calculate the distance between the apartments and the fastfood restaurant or supermarket in Durham. And then transfer the distance into a score as a feature to critize the convenience of the apartment.

### 1.1 Observe the data
We should observe the data firstly and find 
- How big the dataset is
- What columns we could use
- If we have some missing data

In [3]:
# Load the three CSV files and rename it
supermarket = pd.read_csv('Supermarkets2C_Warehouse_Clubs_and_Supercenters.csv') 
fastfood = pd.read_csv('fastfoodmaps_locations_2007.csv', header=None) 
apartment_c = pd.read_excel('Durham_Apartment_Dataset.xlsx') 

Read in the data and display the head to see what data we have, and the shape of dataset

In [4]:
# Read the supermarket dataset
display(supermarket.shape)
display(supermarket.head())
display(supermarket.isna().sum())

(47, 27)

,X,Y,OBJECTID,facilityid,facilityarea,name,factype,fulladdr,municipality,agency,...,operhours,adultmeals,readytoeatmeals,comments,GlobalID,created_user,created_date,last_edited_user,last_edited_date,snapauthorized
0,-78.946860,36.020200,639,147111.0,0,LAGOS MEAT MARKET LLC,"Supermarket, Warehouse Club, Supercenter / Sup...",3438 HILLSBOROUGH RD # C,DURHAM,NaN,...,8:30am - 9 pm,NaN,NaN,NaN,{178A0DF6-368A-4E30-B3A7-6FFD87A1D778},A1,2021/10/20 09:06:17+00,A1,2021/10/20 09:06:17+00,NaN
1,-78.916600,35.996430,678,10818.0,0,DURHAM CO-OP MARKET,"Supermarket, Warehouse Club, Supercenter / Sup...",1111 W CHAPEL HILL ST,DURHAM,NaN,...,9 AM- 9 PM,NaN,NaN,Become an owner for $15 and get 20% off every ...,{E781F267-1031-4386-9B91-6BBA2CA6ADBC},A1,2021/10/20 09:06:17+00,A1,2021/10/20 09:06:17+00,Yes
2,-78.915710,35.997130,679,99501.0,0,Al-TAIBA MARKET,"Supermarket, Warehouse Club, Supercenter / Sup...",1008 W CHAPEL HILL ST,DURHAM,NaN,...,9am-9pm,NaN,NaN,NaN,{9EFDD02D-9B1C-4896-87C1-FB943EEDE1A9},A1,2021/10/20 09:06:17+00,A1,2021/10/20 09:06:17+00,Yes
3,-78.953079,35.903931,861,144685.0,0,TARGET,"Supermarket, Warehouse Club, Supercenter / Sup...",8210 RENAISSANCE PKWY,DURHAM,NaN,...,7 am - 10pm,NaN,NaN,NaN,{91C3B7C9-A1D8-4E0D-953F-A2FF0A044FCE},A1,2021/10/20 09:06:17+00,A1,2021/10/20 09:06:17+00,Yes
4,-78.998642,35.954521,913,141262.0,0,WALMART SUPERCENTER,"Supermarket, Warehouse Club, Supercenter / Sup...",5450 NEW HOPE COMMONS DR,DURHAM,NaN,...,6 am -11 pm,NaN,NaN,NaN,{EF52F3C4-039C-4DF0-9935-265FBD99CE06},A1,2021/10/20 09:06:17+00,A1,2021/10/20 09:06:17+00,Yes


X                    0
Y                    0
OBJECTID             0
facilityid           7
facilityarea         0
name                 0
factype              0
fulladdr             0
municipality         7
agency              47
agencytype          47
agencyurl            7
pocname             47
pocphone            10
pocemail            47
phone                0
operdays             0
operhours            1
adultmeals          47
readytoeatmeals     47
comments            45
GlobalID             0
created_user         0
created_date         0
last_edited_user     0
last_edited_date     0
snapauthorized       3
dtype: int64

In [5]:
# Read the fastfood dataset
fastfood.columns =['sequence', 'Alphabet', 'Address', 'City', 'State', 'Zip Code', 'Phone', 'latitude', 'Longitude','Unknown']
display(fastfood.shape)
display(fastfood.head())
display(fastfood.isna().sum())

(50002, 10)

,sequence,Alphabet,Address,City,State,Zip Code,Phone,latitude,Longitude,Unknown
0,1,b,3601 N.W. 27th Avenue,Miami,FL,33142,(305) 638-3838,25.8092,-80.2400,0
1,2,b,8995 N. W. 7th Avenue,Miami,FL,33150,(305) 754-8453,25.8587,-80.2094,0
2,3,b,30390 South Dixie Highway,Homestead,FL,33030,(305) 247-7181,25.4849,-80.4610,0
3,4,b,7975 N. W. 27th Avenue,Miami,FL,33147,(305) 836-8152,25.8471,-80.2415,0
4,5,b,9201 South Dixie Highway,Miami,FL,33156,(305) 666-1130,25.6849,-80.3125,0


sequence         0
Alphabet         0
Address          0
City             0
State            0
Zip Code         0
Phone        20025
latitude         0
Longitude        0
Unknown          0
dtype: int64

In [6]:
# Read the apartment dataset
display(apartment_c.shape)
display(apartment_c.head())
display(apartment_c.isna().sum())

(25, 5)

,Apartment Name,Address,Latitude,Longitude,Mean Price(2b2b)
0,CHAPEL TOWERS,"1315 MORREENE RD, DURHAM, NC 27705",36.005001,-78.952447,1435
1,BLUE LIGHT LIVING,"1605 ERWIN RD, DURHAM, NC 27705",36.005622,-78.922251,3200
2,CHANDLER AT UNIVERSITY TOWER,"20 MORCROFT LN, DURHAM, NC 27705",35.972722,-78.956174,1650
3,STATION NINE,"2211 HILLSBOROUGH RD, DURHAM, NC 27705",36.009692,-78.925243,2100
4,BRIGHTLEAF ON MAIN,"1005 W MAIN ST, DURHAM, NC 27701",36.001021,-78.910606,2400


Apartment Name      0
Address             0
Latitude            0
Longitude           0
Mean Price(2b2b)    0
dtype: int64

### 1.2 Clean the data
After we observe the data, the next step is we should clean the data for the convenience that we could deal with the data next step. 


About the supermarket data
- We should rename the column, y is latitude, x is longitude, municipality is City
- We want to use the columns latitude, longitude, name, factype, fulladdr and city; even if there are some missing data is this dataset, in these columns, there is no missing data, so that's okay. 





In [7]:
supermarket = supermarket.drop(columns=['OBJECTID', 'facilityid', 'facilityarea', 'agency',
                          'agencytype', 'agencyurl', 'pocname', 'pocphone', 'pocemail',
                          'phone', 'operdays', 'operhours', 'adultmeals', 'readytoeatmeals',
                          'comments', 'GlobalID', 'created_user', 'created_date', 'last_edited_user', 
                          'last_edited_date', 'snapauthorized'])

supermarket = supermarket.rename(columns={"X": "Longitude", "Y": "Latitude", "factype": "type", "fulladdr": "address", "municipality": "city"})


About the fastfood data
- We should delete the column sequence, Alphabet, Phone, Unknown
- Select all the fastfood location in Durham

In [8]:
fastfood = fastfood.drop(columns=['sequence', 'Alphabet', 'Phone', 'Unknown'])
fastfood = fastfood.loc[fastfood['City'] == 'Durham']


About the apartment data
- We just need to delete the unknown column, others are looking good
- We just need to rename the mean price(2b2b) column into mean_price_twoBtwoB, because 2b2b is invalid syntax

In [9]:
#apartment_c = apartment_c.drop(columns=['Unnamed: 5'])
apartment_c = apartment_c.rename(columns={"Mean Price(2b2b)": "mean_price_twoBtwoB"})


# Part 2: Data Processing
### 2.1 Process the distance between fastfood location and apartments
Usually, we choose to walk to fastfood location to buy some convenient food. According to the https://www.healthline.com/health/exercise-fitness/average-walking-speed#:~:text=A%20walking%20speed%20of%203,%2C%20overall%20health%2C%20and%20age, the average speed of an adult is 3 to 4 miles per hour. We just take the 3.5 miles per hour and define:
- "Close distance fastfood", which is between 0 and 0.94 km (we need walk 10 minutes to arrive the fastfood location. Distance = 3.5*1.60934/6 kilometers.)
- "Acceptable distance fastfood", which is between 0.94 and 1.88 km (we need walk 20 minutes to arrive the fastfood location. Distance = 3.5*1.60934/3 kilometers.)
- "Long distance fastfood", which is between 1.88 and 2.82 km (we need walk less than 30 minutes to arrive the fastfood location.)

So, the next step we should calculate the distance between each apartments and each fastfood and figure out for every apartments, how many fastfood location is at close distance, how many fastfood location is at acceptable distance, how many fastfood location is at long distance. We will add the distances to the next columns of fastfood table. And the alogrithm we calculate the distance using latitude and longitude is from https://www.geeksforgeeks.org/program-distance-two-points-earth/

In [10]:
# This is the function that calculate the distance between two points, which are using Latitude and Longitude
def distance(lat1, lat2, lon1, lon2):
     
    # The math module contains a function named
    # radians which converts from degrees to radians.
    lon1 = radians(lon1)
    lon2 = radians(lon2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
      
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
 
    c = 2 * asin(sqrt(a))
    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371
    
      
    # calculate the result
    return(c * r)

# Transfer the data in Latitude and Longitude columns of apartment dataset into list, 
# so that we can take the value easily to do the calculation

apartment_latitude_list = apartment_c['Latitude'].tolist()
apartment_longitude_list = apartment_c['Longitude'].tolist()

# Create a list i, which has 25 elements, so that we can create the 25 new columns and store the distance data we calculate 
i = np.arange(1,26)

# Use "for" syntax to put the distance data we calculate into the new columns we create
for a, b, c in zip(apartment_latitude_list, apartment_longitude_list, i):
    fastfood[c] = fastfood.apply(lambda row: distance(row['latitude'], a, row['Longitude'], b), axis=1)

# show the whole table and check it
display(fastfood.head())



,Address,City,State,Zip Code,latitude,Longitude,1,2,3,4,...,16,17,18,19,20,21,22,23,24,25
214,1200 West Club Boulevard,Durham,NC,27701,36.0171,-78.9120,3.878838,1.574462,6.335931,1.448203,...,3.109606,3.535526,3.015757,3.324450,15.735343,8.577692,11.112446,3.409917,2.242299,1.541985
2803,3520 Hillsborough Road,Durham,NC,27705,36.0215,-78.9509,1.839864,3.123637,5.444611,2.654977,...,1.345057,1.259071,1.516943,1.154523,17.439758,7.604888,10.923349,4.469699,4.535261,3.677392
2991,"3823 Guess Road, Willowdaile Plaza",Durham,NC,27705,36.0198,-78.9589,1.744918,3.653987,5.240610,3.229074,...,1.608109,1.316039,1.794979,1.341527,17.601916,7.307607,10.735981,4.759504,5.030427,4.420942
3293,3400 Westgate Drive,Durham,NC,27707,35.9672,-78.9613,4.278097,5.531483,0.767972,5.731406,...,4.989772,4.879880,4.905984,5.045966,12.913478,1.527087,4.916974,4.099255,5.634546,8.376664
3742,4709 Highway 55 East,Durham,NC,27713,35.9141,-78.9203,10.513712,10.178322,7.274584,10.638673,...,10.925874,11.021560,10.771099,11.101478,6.167737,6.189859,3.257501,8.212462,9.255300,13.017781


### 2.2 Calculate the score of every apartments
We define if a apartment has one fastfood in close distance, it will get 5 points; one fastfood in acceptable distance, it will get 3 points;  one fastfood in long distance, it will get 1 points.
So, next we should find the amount of these three foastfood locations of every apartments and calaulate the score.

In [11]:
# Create a copy of apartment dataset
new_df = apartment_c.copy()


# We create three new lists to store the number of the fastfood locations after we use "loc" syntax to filter
# the fastfood locations that meet our standard. "FF" is the abbreviation of "fastfood"  
all_close_FF = []
for a in i:
    close = fastfood.loc[(fastfood[a] > 0) & (fastfood[a] <= 0.94)]
    number = close.shape[0]
    all_close_FF.append(number) 

all_acceptable_FF = []
for a in i:
    acceptable = fastfood.loc[(fastfood[a] > 0.94) & (fastfood[a] <= 1.88)]
    number = acceptable.shape[0]
    all_acceptable_FF.append(number) 
    
all_long_FF = []
for a in i:
    long = fastfood.loc[(fastfood[a] > 1.88) & (fastfood[a] <= 2.82)]
    number = long.shape[0]
    all_long_FF.append(number) 

# Check all the three lists
print(all_close_FF)
print(all_acceptable_FF)
print(all_long_FF)


# Add the three lists to the apartment dataset
new_df['close_number_FF'] = all_close_FF
new_df['acceptable_number_FF'] = all_acceptable_FF
new_df['long_number_FF'] = all_long_FF

# Then we calculate the score of each apartments
new_df['score_of_fastfood'] = new_df['close_number_FF']*5 + new_df['acceptable_number_FF']*3 + new_df['long_number_FF']*1

# We use the percentile method to change the values so they fall in a 0 to 1 range in descending order according to their score, 
# and multiply 100 to get a score in the form of 100

new_df['Percentile Rank_fastfood'] = new_df.score_of_fastfood.rank(pct = True)
new_df['Percentile Rank_fastfood'] = new_df['Percentile Rank_fastfood']*100

# Display the new table
display(new_df.head())

[0, 0, 3, 0, 0, 2, 2, 4, 0, 4, 3, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[4, 2, 3, 3, 4, 2, 3, 2, 4, 0, 1, 2, 0, 5, 3, 5, 6, 6, 6, 2, 3, 0, 1, 4, 3]
[2, 4, 0, 3, 4, 1, 0, 0, 2, 0, 3, 4, 0, 0, 3, 1, 1, 1, 1, 0, 3, 1, 4, 1, 7]


,Apartment Name,Address,Latitude,Longitude,mean_price_twoBtwoB,close_number_FF,acceptable_number_FF,long_number_FF,score_of_fastfood,Percentile Rank_fastfood
0,CHAPEL TOWERS,"1315 MORREENE RD, DURHAM, NC 27705",36.005001,-78.952447,1435,0,4,2,14,42.0
1,BLUE LIGHT LIVING,"1605 ERWIN RD, DURHAM, NC 27705",36.005622,-78.922251,3200,0,2,4,10,14.0
2,CHANDLER AT UNIVERSITY TOWER,"20 MORCROFT LN, DURHAM, NC 27705",35.972722,-78.956174,1650,3,3,0,24,96.0
3,STATION NINE,"2211 HILLSBOROUGH RD, DURHAM, NC 27705",36.009692,-78.925243,2100,0,3,3,12,28.0
4,BRIGHTLEAF ON MAIN,"1005 W MAIN ST, DURHAM, NC 27701",36.001021,-78.910606,2400,0,4,4,16,50.0


### 2.3 Process the distance between supermarkets, groceries and apartments
Usually, we will choice to drive to supermarket because we need to carry some goods. According to the https://connect.ncdot.gov/resources/safety/Teppl/TEPPL%20All%20Documents%20Library/S-40_Brochure.pdf, the statutory speed limit is 35 miles per hour (MPH) unless otherwise ordinanced and posted. The statutory speed limit on roads outside incorporated municipalities is 55 MPH unless otherwise ordinanced and posted. 
We just take 45 miles per hour (the average of 35 and 55 miles per hour) and define:
- "Close distance supermarket", which is between 0 and 7.5 km (we need drive 10 minutes to arrive the supermarkets or groceries. Distance = 45/6 kilometers.)
- "Acceptable distance supermarket", which is between 7.5 and 15 km (we need drive 20 minutes to arrive the supermarkets or groceries. Distance = 45/3 kilometers.)
- "Long distance supermarket", which is between 15 and 22.5 km (we need drive less than 30 minutes to arrive the supermarkets or groceries.)

So, like the above steps, we should calculate the distance between each apartments and each supermarkets and figure out for every apartments, how many supermarkets is at close distance, how many supermarkets is at acceptable distance, how many supermarkets is at long distance. We will add the distances to the next columns of supermarket table. And the alogrithm we calculate the distance using latitude and longitude is from https://www.geeksforgeeks.org/program-distance-two-points-earth/

In [12]:
# In the same way, get all the distances and store them into the supermarket dataset
for a, b, c in zip(apartment_latitude_list, apartment_longitude_list, i):
    supermarket[c] = supermarket.apply(lambda row: distance(row['Latitude'], a, row['Longitude'], b), axis=1)
    
display(supermarket.head())


,Longitude,Latitude,name,type,address,city,1,2,3,4,...,16,17,18,19,20,21,22,23,24,25
0,-78.946860,36.020200,LAGOS MEAT MARKET LLC,"Supermarket, Warehouse Club, Supercenter / Sup...",3438 HILLSBOROUGH RD # C,DURHAM,1.763171,2.743540,5.345427,2.268330,...,1.116712,1.157433,1.266398,0.993573,17.152027,7.545041,10.796333,4.155379,4.159314,3.379388
1,-78.916600,35.996430,DURHAM CO-OP MARKET,"Supermarket, Warehouse Club, Supercenter / Sup...",1111 W CHAPEL HILL ST,DURHAM,3.362618,1.141544,4.430401,1.667094,...,3.018770,3.400904,2.837321,3.295935,13.666262,6.547952,8.821960,1.138338,0.541124,3.865195
2,-78.915710,35.997130,Al-TAIBA MARKET,"Supermarket, Warehouse Club, Supercenter / Sup...",1008 W CHAPEL HILL ST,DURHAM,3.418708,1.112593,4.541088,1.639081,...,3.050103,3.439012,2.870423,3.327712,13.713082,6.659592,8.924884,1.247720,0.458136,3.779430
3,-78.953079,35.903931,TARGET,"Supermarket, Warehouse Club, Supercenter / Sup...",8210 RENAISSANCE PKWY,DURHAM,11.238626,11.643049,7.654250,12.024034,...,11.838059,11.817436,11.716641,11.950909,8.287631,5.769969,2.159489,9.707290,11.028339,14.607799
4,-78.998642,35.954521,WALMART SUPERCENTER,"Supermarket, Warehouse Club, Supercenter / Sup...",5450 NEW HOPE COMMONS DR,DURHAM,6.984649,8.918327,4.324872,9.014031,...,7.823703,7.561621,7.804932,7.775395,14.520286,2.790005,5.266611,7.705462,9.220144,11.550341


### 2.4 Calculate the score of every apartments
We define if a apartment has one supermarket in close distance, it will get 5 points; one supermarket in acceptable distance, it will get 3 points;  one supermarket in long distance, it will get 1 points.
So, next we should find the amount of these three supermarket locations of every apartments and calaulate the score.

In [13]:
# We create three new lists to store the number of the supermarkets after we use "loc" syntax to filter
# supermarkets that meet our standard. "SM" is the abbreviation of "supermarket" 
all_close_SM = []
for a in i:
    close = supermarket.loc[(supermarket[a] > 0) & (supermarket[a] <= 7.5)]
    number = close.shape[0]
    all_close_SM.append(number) 

all_acceptable_SM = []
for a in i:
    acceptable = supermarket.loc[(supermarket[a] > 7.5) & (supermarket[a] <= 15)]
    number = acceptable.shape[0]
    all_acceptable_SM.append(number) 
    
all_long_SM = []
for a in i:
    long = supermarket.loc[(supermarket[a] > 15) & (supermarket[a] <= 22.5)]
    number = long.shape[0]
    all_long_SM.append(number) 

# Check all the three lists
print(all_close_SM)
print(all_acceptable_SM)
print(all_long_SM)

# Add the three lists to the apartment dataset
new_df['close_number_SM'] = all_close_SM
new_df['acceptable_number_SM'] = all_acceptable_SM
new_df['long_number_SM'] = all_long_SM

# Then we calculate the score of each apartments
new_df['score_of_supermarket'] = new_df['close_number_SM']*5 + new_df['acceptable_number_SM']*3 + new_df['long_number_SM']*1

# We use the percentile method to change the values so they fall in a 0 to 1 range in descending order according to their score, 
# and multiply 100 to get a score in the form of 100

new_df['Percentile Rank_supermarket'] = new_df.score_of_supermarket.rank(pct = True)
new_df['Percentile Rank_supermarket'] = new_df['Percentile Rank_supermarket']*100




[29, 29, 26, 29, 31, 30, 23, 26, 23, 30, 31, 26, 30, 27, 22, 28, 27, 28, 28, 4, 22, 15, 26, 31, 26]
[16, 17, 20, 17, 15, 17, 22, 20, 22, 17, 16, 20, 17, 19, 22, 17, 18, 17, 17, 26, 23, 25, 20, 15, 18]
[2, 1, 1, 1, 1, 0, 2, 1, 2, 0, 0, 1, 0, 1, 3, 2, 2, 2, 2, 17, 2, 7, 1, 1, 3]


In [14]:
# Display the new table
display(new_df.head())

,Apartment Name,Address,Latitude,Longitude,mean_price_twoBtwoB,close_number_FF,acceptable_number_FF,long_number_FF,score_of_fastfood,Percentile Rank_fastfood,close_number_SM,acceptable_number_SM,long_number_SM,score_of_supermarket,Percentile Rank_supermarket
0,CHAPEL TOWERS,"1315 MORREENE RD, DURHAM, NC 27705",36.005001,-78.952447,1435,0,4,2,14,42.0,29,16,2,195,68.0
1,BLUE LIGHT LIVING,"1605 ERWIN RD, DURHAM, NC 27705",36.005622,-78.922251,3200,0,2,4,10,14.0,29,17,1,197,74.0
2,CHANDLER AT UNIVERSITY TOWER,"20 MORCROFT LN, DURHAM, NC 27705",35.972722,-78.956174,1650,3,3,0,24,96.0,26,20,1,191,40.0
3,STATION NINE,"2211 HILLSBOROUGH RD, DURHAM, NC 27705",36.009692,-78.925243,2100,0,3,3,12,28.0,29,17,1,197,74.0
4,BRIGHTLEAF ON MAIN,"1005 W MAIN ST, DURHAM, NC 27701",36.001021,-78.910606,2400,0,4,4,16,50.0,31,15,1,201,88.0


### 2.5 Calculate the score of every apartments in price
Price is another factor students will consider a lot when they choose apartments. So, as the same way of fastfood and groceries, we use percentile method to give score to each apartment based on price. The lower the price is, the higher the score will be.

In [15]:
# use percentile ranking, set the parameter ascending = false
new_df['Percentile Rank_price'] = new_df.mean_price_twoBtwoB.rank(pct = True, ascending = False)
new_df['Percentile Rank_price'] = new_df['Percentile Rank_price']*100

### 2.6 Calculate the score of every apartments based on the distance to university
Also, the distance to Duke University matters the desicion of the students. If the apartments are close to the campus, students will choose to walk to school; if the apartments are far away from the campus, students will consider buying a car. We use percentile to calculate the score of each apartments based on the distance to the university. The smaller the distance is, the higher the score will be.

In [16]:
# This is the coordinate of Duke University, the coordinate comes from Google Map
Latitude_duke = 36.00163117620302
Longitude_duke = -78.93824053356649

# Using the distance function to calculate the distances of each apartment
new_df['distance_to_university'] = new_df.apply(lambda row: distance(row['Latitude'], Latitude_duke, row['Longitude'], Longitude_duke), axis=1)
new_df['distance_to_university'] = new_df['distance_to_university'].round(decimals = 2)

# use percentile ranking, set the parameter ascending = false
new_df['Percentile Rank_university'] = new_df.distance_to_university.rank(pct = True, ascending = False)
new_df['Percentile Rank_university'] = new_df['Percentile Rank_university']*100


In [17]:
display(new_df.head())

,Apartment Name,Address,Latitude,Longitude,mean_price_twoBtwoB,close_number_FF,acceptable_number_FF,long_number_FF,score_of_fastfood,Percentile Rank_fastfood,close_number_SM,acceptable_number_SM,long_number_SM,score_of_supermarket,Percentile Rank_supermarket,Percentile Rank_price,distance_to_university,Percentile Rank_university
0,CHAPEL TOWERS,"1315 MORREENE RD, DURHAM, NC 27705",36.005001,-78.952447,1435,0,4,2,14,42.0,29,16,2,195,68.0,68.0,1.33,92.0
1,BLUE LIGHT LIVING,"1605 ERWIN RD, DURHAM, NC 27705",36.005622,-78.922251,3200,0,2,4,10,14.0,29,17,1,197,74.0,4.0,1.51,76.0
2,CHANDLER AT UNIVERSITY TOWER,"20 MORCROFT LN, DURHAM, NC 27705",35.972722,-78.956174,1650,3,3,0,24,96.0,26,20,1,191,40.0,56.0,3.60,40.0
3,STATION NINE,"2211 HILLSBOROUGH RD, DURHAM, NC 27705",36.009692,-78.925243,2100,0,3,3,12,28.0,29,17,1,197,74.0,42.0,1.47,80.0
4,BRIGHTLEAF ON MAIN,"1005 W MAIN ST, DURHAM, NC 27701",36.001021,-78.910606,2400,0,4,4,16,50.0,31,15,1,201,88.0,22.0,2.49,68.0


#  Asmita Dalai's coding part


### 3.1 Crime Data Preparation
We now take into account the crime factor in calculating the score of the apartment. The data is a csv file downloaded from the site https://durhamnc.gov/149/Police-Department. It consists of the entire Durham's police stations reported cases for 3 years (2018-2020) . Our purpose is to classify the crimes into levels of seriousness and calculate a crime score for each street near Duke University and merge it with Apartment dataset.

In [18]:
##Load the Durham crime dataset 
df = pd.read_excel('Durham_crime_data.xlsx') 

display(df.shape)
display(df.head())

(78368, 15)

,Case Number,Report Date,Report Time,Status,Sequence,ATT/COM,UCR Code,Offense,Address,X,Y,District,Beat,Premise,Weapon
0,20042658,2020-12-31,12:27:00AM,Cleared By Arrest,1,COM,290,VANDALISM,3500 PLAY GATE LN,2053032,816540,D1,123,Home Of Offender - Single Family,(blank)
1,20042661,2020-12-31,12:01:00AM,Closed (Non-Criminal),1,COM,9925,TOWED/ABANDONED VEHICLE,2800 COURTNEY CREEK BLVD,2032391,796701,D4,414,Parking/Drop Lot/Garage,(blank)
2,20042664,2020-12-31,12:49:00AM,Closed (Non-Criminal),1,COM,9925,TOWED/ABANDONED VEHICLE,2800 COURTNEY CREEK BLVD,2032400,796703,D4,414,Parking/Drop Lot/Garage,(blank)
3,20042666,2020-12-31,2:03:00AM,Active/Open,1,COM,13A,AGGRAVATED ASSAULT,3400 EMILY ST,2034727,831973,D2,214,Home Of Victim - Single Family,Fire/Burning Tool/Device
4,20042666,2020-12-31,2:03:00AM,Active/Open,2,COM,13B,SIMPLE ASSAULT,3400 EMILY ST,2034758,831987,D2,214,Home Of Victim - Single Family,Not Applicable/None



### 3.2 Data Cleaning and Filtering
We are only considering only the crimes which are active , cleared by arrest (someone was arrested) and cleared by exception
Exceptions are deffined as : The crime where elements beyond law enforcement’s control prevent the agency from arresting 
and formally charging the offender. But the crime is valid

In [19]:
crimes_to_keep = ['Active/Open','Cleared By Arrest','Cleared By Exception']
df_active = df[df.Status.isin(crimes_to_keep) ]
df_active['Status'].value_counts()

Active/Open             19971
Cleared By Arrest       13102
Cleared By Exception     3580
Name: Status, dtype: int64

### 3.3 Crime Level Grouping
North Carolina Uniform Crime Reporting (UCR) Program is part of a nationwide, cooperative statistical effort administered by the Federal Bureau of Investigation. UCR Code list is a fixed offense code list assigned to each crime reported. We use UCR Code column to group our crimes into different level of seriousness. 1 is the least serious and 5 the most serious.
You can refer to the site to get the crime associated with each UCR code (https://apps.ncsc.org/wayfarer1.9/codetables/OffenseCodeType.asp). 

In [20]:
## Use UCR codes to group the crimes together at levels of seriousness. 
## UCR codes definition : A code set that enumerates Uniform Crime Reporting (UCR) offense codes.
Crime_level_1 = ['270','26A','26B','26C','26D','26E','35A','35B','370','39A','39B','39C','39D','510','90B','90C','90G','90H']
Crime_level_2 = ['250' ,'520' ,'90D' ,'90F','290']
Crime_level_3 = ['220','240','90Z','90E','23C','23E','23G','23H']
Crime_level_4 = ['120','200','90J','13A','13B','13C','36A','36B','40A','40B', '23A','23B','23D','23F']
Crime_level_5 = ['100','11A','11B','11C','11D','280','90A','90I']

In [21]:
### Changing the UCR data type to string and trimming any spaces
df_active['Crime_level'] = 0
df_active = df_active.rename(columns={"UCR Code": "UCR_Code"})
df_active['UCR_Code'] = df_active['UCR_Code'].apply(str)
df_active['UCR_Code'] = df_active['UCR_Code'].str.strip()
df_active['UCR_Code'].value_counts()

13B     3407
290     3300
23C     3127
23F     2966
90Z     2300
        ... 
40C        1
9911       1
26D        1
9910       1
26G        1
Name: UCR_Code, Length: 75, dtype: int64

In [22]:
## Mapping cRime level against each crime
df_active.loc[df_active.UCR_Code.isin(Crime_level_1), 'Crime_level'] = 5
df_active.loc[df_active.UCR_Code.isin(Crime_level_2), 'Crime_level'] = 20
df_active.loc[df_active.UCR_Code.isin(Crime_level_3), 'Crime_level'] = 50
df_active.loc[df_active.UCR_Code.isin(Crime_level_4), 'Crime_level'] = 80
df_active.loc[df_active.UCR_Code.isin(Crime_level_5), 'Crime_level'] = 100

##Taking only crimes with level assigned 1-5
df_active = df_active.loc[df_active['Crime_level'] != 0]
df_active['Crime_level'].value_counts()

80     11053
50      9857
20      4995
5       3943
100      544
Name: Crime_level, dtype: int64

### 3.4 Extracting Street from Address
When looking at crime , we map a crime score against a street. We don't want to limit ourselves to a certain distance near the apartment as crimes are very common on the entire street near the apartment. Hence , we use the address to extract the street name.

In [23]:
df_active['Street'] = df_active['Address'].str.replace('\d+', '').str.strip()
df_active['Street'] = df_active['Street'].str.split('/').str[0]
df_near_duke_crimes = df_active[df_active['Street'].str.contains('MORREENE|ERWIN|MOCROFT|Hillsborough|W Main St|Willard St|Alexan Dr|University Dr|S Mangum St|Foster St|Leon St|Garrett Rd|Campus Walk Ave|McQueen Dr|Finsbury St|Hope Valley Rd|Palmer St|W Chapel Hill St|Broad St|Morcroft Ln', flags=re.IGNORECASE, regex=True, na=True)]
df_near_duke_crimes['Street'].value_counts()

HILLSBOROUGH RD     465
UNIVERSITY DR       318
S MANGUM ST         258
W MAIN ST           221
GARRETT RD          186
BROAD ST            148
ERWIN RD            140
HOPE VALLEY RD      138
MORREENE RD          95
W CHAPEL HILL ST     87
LEON ST              59
FOSTER ST            42
WILLARD ST           26
CAMPUS WALK AVE      23
PALMER ST            16
FINSBURY ST          15
ALEXAN DR            10
MCQUEEN DR            9
MORCROFT LN           9
Name: Street, dtype: int64

### 3.5 Calculating Street crime score
We first filter the dataset and select only the columns required (Street name and Crime level). We then group the dataset by Street and take an average Crime level. We then scale the data to get a score out of 100.

In [24]:
df_final = df_near_duke_crimes[['Street','Crime_level']]
df_street_rating = pd.DataFrame(df_final.groupby(['Street']).sum())
df_crime_score = df_street_rating.reset_index()
display(df_crime_score)

,Street,Crime_level
0,ALEXAN DR,760
1,BROAD ST,7855
2,CAMPUS WALK AVE,1255
3,ERWIN RD,7585
4,FINSBURY ST,750
5,FOSTER ST,2360
6,GARRETT RD,10105
7,HILLSBOROUGH RD,23750
8,HOPE VALLEY RD,7695
9,LEON ST,3080


In [25]:
new_df['Street'] = new_df['Address'].str.replace('\d+', '').str.strip()
new_df['Street'] = new_df['Street'].str.split(',').str[0]
new_df['Street'].value_counts()


UNIVERSITY DR       4
ERWIN RD            3
S MANGUM ST         2
CAMPUS WALK AVE     1
PALMER ST           1
MORCROFT LN         1
MORREENE RD         1
BROAD ST            1
HILLSBOROUGH RD     1
W CHAPEL HILL ST    1
MCQUEEN DR          1
FOSTER ST           1
ALEXAN DR           1
W MAIN ST           1
LEON ST             1
WILLARD ST          1
GARRETT RD          1
HOPE VALLEY RD      1
FINSBURY ST         1
Name: Street, dtype: int64

In [26]:
df = new_df.merge(df_crime_score, how = 'inner', right_on = 'Street' , left_on = 'Street')
#df = df.drop(columns=['Crime_level'])
df['Percentile_crime_rank'] = df.Crime_level.rank(pct = True, ascending = False)
df['Percentile_crime_rank'] = df['Percentile_crime_rank']*100


In [27]:
df['Percentile_crime_rank'] = df['Percentile_crime_rank'].astype(int)

#  Arrange our final output

In [28]:
df_final = df.copy()

# Delete some unnecessary columns
df_final = df_final.drop(columns=['close_number_FF', 'acceptable_number_FF', 
                                  'long_number_FF', 'score_of_fastfood',
                                 'close_number_SM', 'acceptable_number_SM','long_number_SM',
                                 'score_of_supermarket', 'Street', 'distance_to_university', 'Crime_level',
                                 'Latitude', 'Longitude'])
# The weight of each factor in formula below is used to calculate the personalized score
#df_final['final_score'] = 0.33*df_final['Percentile_crime_rank'] + 0.2*df_final['Percentile Rank_price'] + 0.27*df_final['Percentile Rank_university'] + 0.07*df_final['Percentile Rank_fastfood'] + 0.13*df_final['Percentile Rank_supermarket']

# The weight of each factor in formula below is decided by our first survey: how students think about the priority of these factors
df_final['final_score'] = 0.332*df_final['Percentile_crime_rank'] + 0.308*df_final['Percentile Rank_price'] + 0.176*df_final['Percentile Rank_university'] + 0.08*df_final['Percentile Rank_fastfood'] + 0.104*df_final['Percentile Rank_supermarket']


# sort and rename the column and give the final ranking
df_final = df_final.sort_values(by='final_score', ascending=False)
df_final.columns =['Apartment Name', 'Address', 'Price 2b2b', 'Fastfood Score', 'Supermarket Score', 'Price Score', 'Campus Distance Score', 'Crime Score', 'Final Score']

df_final.head()





,Apartment Name,Address,Price 2b2b,Fastfood Score,Supermarket Score,Price Score,Campus Distance Score,Crime Score,Final Score
19,THE BELMONT APARTMENTS,"1000 MCQUEEN DR, DURHAM, NC 27705",1400,66.0,58.0,76.0,88.0,96,82.080
18,CAMPUS WALK APARTMENT COMPLEX,"2748 CAMPUS WALK AVE, DURHAM, NC 27705",1000,66.0,40.0,96.0,84.0,80,80.352
22,WILLOW BEND APARTMENTS,"1718 PALMER ST, DURHAM, NC 27707",1000,8.0,40.0,96.0,72.0,84,74.928
4,CHANDLER AT UNIVERSITY TOWER,"20 MORCROFT LN, DURHAM, NC 27705",1650,96.0,40.0,56.0,40.0,100,69.328
0,CHAPEL TOWERS,"1315 MORREENE RD, DURHAM, NC 27705",1435,42.0,68.0,68.0,92.0,60,67.488


In [29]:
# output a xlsx file
file_name = 'finalscore.xlsx'
df_final.to_excel(file_name)